<p style="text-align:center">
  <a href="https://www.linkedin.com/company/100622063" target="_blank" title="Follow LevelUp360 on LinkedIn">
    <img src="../../assets/levelup360-inverted-logo-transparent.svg" alt="LevelUp360" width="220">
  </a>
</p>

# Marketing Team – Week 06: Agentic Content Generation Workflow (Microsoft Agent Framework)

This notebook validates the full agentic workflow implemented in Week 6 using the **Microsoft Agent Framework**:

**content_planning → tools → content_generation → content_evaluation → (loop or end)**

We use a content planning executor to decide tools (RAG, Web, both, none), dedicated tool executors to fetch contexts, a generation executor that synthesizes the user prompt and context retrireved by the tool executor (if any) into a draft, and an evaluation executor that scores quality and controls a regeneration loop (iteration_count vs max_iterations, threshold).

---

## What We’re Testing

- Tool routing: content planning executor chooses `rag_search`, `web_search`, both, or none
- Agentic execution: tool executors fetch contexts; generation consumes tool outputs (no duplicate calls)
- Draft creation: generation executor builds prompts via brand/template config
- Evaluation & loop: evaluation executor scores content; loop control fields on the shared state

**Architecture:** Microsoft Agent Framework Workflow → Executors:

- `StartExecutor` → seeds `ContentThreadState` from a user topic
- `ContentPlanningExecutor` → decides tools and records `planning_decision`
- Tool path (RAG / Web / both / none) → fills `tool_contexts` / research fields
- `ContentGenerationExecutor` → generates draft content into `state.content`
- `ContentEvaluationExecutor` → critiques draft, updates loop control fields
- `FinalStateExecutor` → yields final `ContentThreadState` as workflow output

---

## Environment Setup

### Prerequisites
- Python 3.10+
- `.env` with API keys (see below)
- Virtual environment in repo root
- VS Code Jupyter extension (or JupyterLab)

### Required Environment Variables
    # OpenRouter / Azure / other model provider
    OPENROUTER_API_KEY=sk-...              # or provider-specific key

    # Azure (optional for embeddings / models)
    AZURE_OPENAI_ENDPOINT=https://...
    AZURE_OPENAI_API_KEY=...
    AZURE_OPENAI_API_VERSION=...

    # Web search
    TAVILY_API_KEY=...

    # LangSmith (optional tracing)
    LANGSMITH_TRACING_V2=true
    LANGSMITH_API_KEY=...
    LANGSMITH_PROJECT=...
    LANGSMITH_ENDPOINT=https://api.smith.langchain.com

    # (Optional) Pricing for local cost tracking
    GPT4O_INPUT_PRICE_PER_1K=...
    GPT4O_OUTPUT_PRICE_PER_1K=...
    EMBEDDING_PRICE_PER_1K=...
    TAVILY_PRICE_PER_CALL=...

### One-Time Setup (PowerShell on Windows)
    # From workspace root:
    python -m venv .venv
    .\\.venv\\Scripts\\Activate.ps1

    # If activation blocked, run once (PowerShell):
    Set-ExecutionPolicy -Scope CurrentUser RemoteSigned

    # Upgrade tooling
    python -m pip install --upgrade pip setuptools wheel

    # Install dependencies (root project + marketing example)
    pip install -e .
    pip install -r examples/marketing_team/requirements.txt

    # (Optional) Register kernel
    python -m ipykernel install --user --name marketing-team --display-name "Python (marketing-team)"

### One-Time Setup (macOS/Linux bash)
    # From workspace root:
    python -m venv .venv
    source .venv/bin/activate

    # Upgrade tooling
    python -m pip install --upgrade pip setuptools wheel

    # Install dependencies
    pip install -e .
    pip install -r examples/marketing_team/requirements.txt

    # (Optional) Register kernel
    python -m ipykernel install --user --name marketing-team --display-name "Python (marketing-team)"

### Verify Installation
    # Should import the Agent Framework workflow without error
    python -c "from src.orchestration.microsoft_agent_framework.workflows.content_generation_workflow import build_content_generation_workflow; print('OK')"

---

## Notebook Flow

1. **Setup** – Autoreload, env, logging  
2. **Build Workflow** – `build_content_generation_workflow(brand)` compiles the Microsoft Agent Framework workflow  
3. **Helpers** – Utilities to run scenarios and render messages, draft, evaluation  
4. **Scenarios** – RAG-only, Web-only, Both, None  
5. **Summary** – Inspect loop stats, metadata, outcomes; capture for documentation

---

## Data & Config

- `examples/marketing_team/configs/` – Brand YAMLs (models, retrieval, formatting, voice, CTA)  
- `examples/marketing_team/data/chroma_db/` – Vector store persistence (RAG)  
- `examples/marketing_team/data/week_06/` – Optional export path for results 

---

## Key Settings Per Run

- `template`: e.g., `LINKEDIN_POST_ZERO_SHOT`, `LINKEDIN_LONG_POST_ZERO_SHOT`, `BLOG_POST`  
- `max_iterations`: int (e.g., 3)  
- `quality_threshold`: float (e.g., 7.0)  
- `use_cot`: bool; adds reasoning scaffold to the prompt (not included in output)  
- `brand`: i.e. `"itconsulting"` or `"aurora"` (from the provided examples)

---

## Expected Outputs

- Message flow captured in `ContentThreadState.messages` with tool routing decisions  
- Draft content (preview) stored in `ContentThreadState.content`  
- Evaluation summary with score, reasoning, violations (if any)  
- Loop stats: `iteration_count`, `max_iterations`, `meets_quality_threshold`  
- Generation/Evaluation metadata (model, cost, latency, tokens) where available

---

## Notes

- Routing evaluator (Week 6 routing tests) remains separate; this notebook focuses on **end-to-end content generation and evaluation loop** using the new workflow.

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
from dotenv import load_dotenv
load_dotenv()

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(name)s | %(message)s'
)
logger = logging.getLogger("week6_agentic_content_generation")
logger.info("Environment & logging initialized (Week 6 – Agent Framework)")

In [ ]:
from typing import Dict, Any, List
from rich import print as rprint

from src.core.utils.config_loader import load_brand_config
from src.orchestration.microsoft_agent_framework.workflows.content_generation_workflow import (
    build_content_generation_workflow,
)
from src.orchestration.microsoft_agent_framework.thread_states.content_thread_state import (
    ContentThreadState,
)

logger.info("Imports loaded (brand config, workflow builder, thread state)")

## Build Workflow (Microsoft Agent Framework)

We build the agentic content workflow for a given brand using the Microsoft Agent Framework.
The compiled workflow executes the full executor graph with a single `run()` call.


In [ ]:
# Choose brand for content generation
brand = "itconsulting"  
brand_config = load_brand_config(brand)

# Build the Microsoft Agent Framework content generation workflow
workflow = build_content_generation_workflow(brand=brand)
logger.info("Microsoft Agent Framework content generation workflow built and compiled")

# Debug: inspect workflow structure
logger.info(f"Workflow executors: {list(workflow.executors.keys())}")
logger.info(f"Workflow edge_groups: {len(workflow.edge_groups)}")
for idx, eg in enumerate(workflow.edge_groups):
    logger.info(f"  EdgeGroup[{idx}]: {type(eg).__name__}, edges={len(eg.edges) if hasattr(eg, 'edges') else 'N/A'}")

## Helpers

Utilities to run a scenario and display the message flow, the generated draft, and the evaluation outcome from the `ContentThreadState` returned by the workflow.


In [ ]:
def _safe_get_thread_state_from_result(result: Any) -> ContentThreadState:
    # Direct: result.state
    if hasattr(result, "state") and isinstance(result.state, ContentThreadState):
        return result.state

    # Primary: result.get_outputs()[0].state
    if hasattr(result, "get_outputs"):
        outs = result.get_outputs()
        if isinstance(outs, list) and outs:
            first = outs[0]
            if hasattr(first, "state") and isinstance(first.state, ContentThreadState):
                return first.state
            # If first is a dict with stored thread namespace
            if isinstance(first, dict):
                ns = first.get("thread") or first.get("state")
                if hasattr(ns, "state") and isinstance(ns.state, ContentThreadState):
                    return ns.state

    # Fallback: outs is a single namespace instead of list
    if hasattr(result, "get_outputs"):
        outs = result.get_outputs()
        if hasattr(outs, "state") and isinstance(outs.state, ContentThreadState):
            return outs.state

    raise RuntimeError("Workflow did not return a ContentThreadState in .state or outputs[0].state")

def print_messages_from_state(state: ContentThreadState) -> None:
    """Print the message flow stored in `state.messages` in a compact form."""
    messages = getattr(state, "messages", None) or []
    print("\n--- Message Flow (ContentThreadState.messages) ---")
    for i, m in enumerate(messages):
        role = m.get("role", "unknown") if isinstance(m, dict) else str(type(m))
        name = None
        if isinstance(m, dict):
            name = m.get("name") or (m.get("metadata") or {}).get("type")
        prefix = f"{i:02d}: {role}"
        if name:
            prefix += f" ({name})"
        content_preview = str(m.get("content", "")) if isinstance(m, dict) else ""
        if len(content_preview) > 80:
            content_preview = content_preview[:77] + "..."
        print(f"{prefix}: {content_preview}")


async def run_scenario(
    topic: str,
    brand: str,
    brand_config: Dict[str, Any],
    template: str = "LINKEDIN_POST_ZERO_SHOT",
    examples: List[str] = [],
    use_cot: bool = False,
    max_iterations: int = 3,
    quality_threshold: float = 7.0,
) -> ContentThreadState:
    """Run the workflow and return the final ContentThreadState."""
    initial_message: Dict[str, Any] = {
        "brand": brand,
        "topic": topic,
        "brand_config": brand_config,
        "template": template,
        "examples": examples,
        "quality_threshold": quality_threshold,
        "use_cot": use_cot,
        "max_iterations": max_iterations,
    }
    result = await workflow.run(message=initial_message)
    thread_state = _safe_get_thread_state_from_result(result)
    return thread_state


def show_result(state: ContentThreadState) -> None:
    """Render a human-readable summary of the final ContentThreadState."""
    print_messages_from_state(state)

    draft = getattr(state, "content", None) or getattr(state, "draft_content", "")
    print("\n[bold green]Final Draft:[/bold green]\n")
    rprint(draft)

    print("\n--- Evaluation ---\n")
    critique = getattr(state, "critique", None) or getattr(state, "evaluation", None)
    if critique:
        score = getattr(critique, "average_score", None) or getattr(
            critique, "score", None
        )
        meets = getattr(state, "meets_quality_threshold", None)
        print(
            f"Score (weighted/average): {( '%.2f' % score) if isinstance(score, (int, float)) else 'N/A'} | "
            f"Meets threshold: {meets}"
        )

        scores_attr = getattr(critique, "scores", None)
        if isinstance(scores_attr, dict):
            print(
                "Dimension scores:",
                {k: f"{v:.2f}" for k, v in scores_attr.items() if isinstance(v, (int, float))},
            )

        reasoning = getattr(critique, "reasoning", None)
        if reasoning:
            print("Reasoning:")
            print(reasoning)

        violations = getattr(critique, "violations", None) or []
        if violations:
            print("Violations:")
            for v in violations:
                print(f"- {v}")
    else:
        print("No critique present (possibly ended before evaluation).")

    print("\n--- Loop Stats ---\n")
    iteration_count = getattr(state, "iteration_count", None)
    max_iterations_val = getattr(state, "max_iterations", None)
    quality_threshold_val = getattr(state, "quality_threshold", None)
    print(
        f"iteration_count={iteration_count} | "
        f"max_iterations={max_iterations_val} | "
        f"quality_threshold={quality_threshold_val}"
    )

    generation_metadata = getattr(state, "generation_metadata", None)
    evaluation_metadata = getattr(state, "evaluation_metadata", None)
    if generation_metadata:
        print("\nGeneration metadata:")
        rprint(generation_metadata)
    if evaluation_metadata:
        print("\nEvaluation metadata:")
        rprint(evaluation_metadata)

## Test 1 — RAG-only scenario (Internal brand content)

Expected: content_planning routes to `rag_search`; tool executors run RAG; generation uses RAG context; evaluation runs; loop ends when threshold met or max iterations reached.


In [ ]:
topic_rag = "Create a post about our Public Cloud governance approach"
state_rag = await run_scenario(
    brand=brand,
    brand_config=brand_config,
    topic=topic_rag,
    template="LINKEDIN_POST_ZERO_SHOT",
    use_cot=True,
    max_iterations=3,
    quality_threshold=7.0,
)
show_result(state_rag)

## Test 2 — Web-only scenario (Current events)

Expected: content_planning routes to `web_search`; generation uses web context.


In [ ]:
topic_web = "Create a post about the latest Public Cloud regulation news in 2025"
state_web = await run_scenario(
    topic_web,
    brand=brand,
    brand_config=brand_config,
    template="LINKEDIN_POST_ZERO_SHOT",
    use_cot=False,
    max_iterations=3,
    quality_threshold=7.0,
)
show_result(state_web)

## Test 3 — Both tools scenario (Internal + Industry)

Expected: content_planning routes to **both** `rag_search` and `web_search`.


In [ ]:
topic_both = "Compare our Public Cloud governance approach to current industry standards in 2025"
state_both = await run_scenario(
    topic_both,
    brand=brand,
    brand_config=brand_config,
    template="LINKEDIN_POST_ZERO_SHOT",
    use_cot=True,
    max_iterations=3,
    quality_threshold=7.0,
)
show_result(state_both)

## Test 4 — No-tools scenario (Opinion / Prompt-only)

Expected: content_planning chooses no tools; generation runs directly from prompt and brand/template configuration.


In [ ]:
topic_none = "Share my opinion on why Public Cloud governance matters for small teams"
state_none = await run_scenario(
    topic_none,
    brand=brand,
    brand_config=brand_config,
    template="LINKEDIN_POST_ZERO_SHOT",
    use_cot=False,
    max_iterations=3,
    quality_threshold=7.0,
)
show_result(state_none)

## Optional Long-Form Scenario (Brand Journal-style Post)

In [ ]:
long_form_topic = """

"""

state_long = await run_scenario(
    long_form_topic,
    brand=brand,
    brand_config=brand_config,
    template="BLOG_POST",
    use_cot=False,
    max_iterations=3,
    quality_threshold=7.0,
)
show_result(state_long)



## Summary

- content_planning (Microsoft Agent Framework executor) routed correctly (RAG / Web / Both / None)  
- tool executors populated research/tool contexts consumed by the generation executor  
- content_evaluation executor returned a critique object, updated `iteration_count`, and set `meets_quality_threshold`  
- Conditional routing (implemented inside the workflow graph) looped to regeneration when threshold was unmet and iterations remained  
- Overall behavior should match the Week 4 LangGraph workflow, but using the Microsoft Agent Framework runtime and executors.
